In [1]:
import pandas as pd
import numpy as np
import time
import ee
import pickle
ee.Initialize()

In [2]:
PATCH_SIZE = 100

# Set up bounding box

In [3]:
def square(lat=45.475649, lon=-69.471018, size=100):
  crs_proj = "EPSG:4326"  
  return ee.Geometry.Point([lon, lat], proj=crs_proj).buffer(size).bounds()

In [4]:
srtm = ee.Image('USGS/SRTMGL1_003')
slope = ee.Terrain.slope(srtm)
jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("seasonality", "recurrence")

def get_stats(image=ee.Image('USGS/SRTMGL1_003'), lat=45.475649, lon=-69.471018, size=100):
    
    try:

        mean = image.reduceRegion(
                reducer = ee.Reducer.mean(),
                geometry = square(lat, lon, size),
                scale = 30,
                maxPixels = 1e9
            ).getInfo()

        stdDev = image.reduceRegion(
                reducer = ee.Reducer.stdDev(),
                geometry = square(lat, lon, size),
                scale = 30,
                maxPixels = 1e9
            ).getInfo()#.get('elevation')

        maxMin = image.reduceRegion(
                reducer = ee.Reducer.minMax(),
                geometry = square(lat, lon, size),
                scale = 30,
                maxPixels = 1e9
        ).getInfo()
        return mean, stdDev, maxMin

    except Exception as e:
        print(e)
        return np.nan, np.nan, np.nan
    



def get_transition(lat=45.475649, lon=-69.471018, size=100):
    try:
        result = ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("transition").reduceRegion(
                    reducer = ee.Reducer.frequencyHistogram(),
                    geometry = square(lat, lon, size),
                    scale = 30,
                    maxPixels = 1e9
            ).getInfo()
        return result.get('transition')
    except Exception as e:
        print(e)
        return np.nan

In [5]:
# read in csv file 
# df_m = pd.read_csv("combined_regular_clean_with_ssurgo_variables.csv")
df_m = pd.read_pickle("cwr_nwpr_dataset")

In [6]:
def extract_image_stats(df_m_, i, size):

  df_m_["srtm_stats"] = (df_m_.apply(lambda x: 
                                      get_stats(image=ee.Image('USGS/SRTMGL1_003'),
                                                lat=x.latitude, 
                                                lon=x.longitude, 
                                                size=size), 
                                      axis=1))
  df_m_["slope_stats"] = (df_m_.apply(lambda x: 
                                      get_stats(image = ee.Terrain.slope(ee.Image('USGS/SRTMGL1_003')),
                                                lat=x.latitude, 
                                                lon=x.longitude, 
                                                size=size), 
                                      axis=1))

  df_m_["seasonality_stats"] = (df_m_.apply(lambda x: 
                                      get_stats(image=ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("seasonality"),
                                                lat=x.latitude, 
                                                lon=x.longitude, 
                                                size=size), 
                                      axis=1))

  df_m_["recurrence_stats"] = (df_m_.apply(lambda x: 
                                      get_stats(image=ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("recurrence"),
                                                lat=x.latitude, 
                                                lon=x.longitude, 
                                                size=size), 
                                      axis=1))
    

  df_m_["transition_hist"] = (df_m_.apply(lambda x: 
                                      get_transition(lat=x.latitude, 
                                                lon=x.longitude, 
                                                size=size), 
                                      axis=1))
  
  # pickle the dataframe 
  (pickle.dump(df_m_, open("cwr_nwpr/" + str(2 * PATCH_SIZE) + "m_SRTM/image_stats_" 
                           + str(2*PATCH_SIZE) + "X" + str(2*PATCH_SIZE)
                           + "_part" + str(i),"wb"), 
               protocol=3))        

In [7]:
from datetime import datetime
print(datetime.now())

2021-04-03 01:33:23.063522


In [8]:
# pass in batches of 500
# MADHUKAR: 0 - 5000 
# SHOBHA: 5000 - 10000
# RADHIKA: 10000 - 15000

patch_size = PATCH_SIZE
batch_size = 50
MY_NAME = "MADHUKAR"
START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

for batch in range(10,65):
  print("batch {} of 10 started".format(batch + 1))
  batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
  extract_image_stats(batch_df, (START + batch_size * batch) + 1, size = patch_size)
  print("batch {} of 10 done".format(batch + 1))
    


batch 11 of 10 started
batch 11 of 10 done
batch 12 of 10 started
batch 12 of 10 done
batch 13 of 10 started
batch 13 of 10 done
batch 14 of 10 started
batch 14 of 10 done
batch 15 of 10 started
batch 15 of 10 done
batch 16 of 10 started
batch 16 of 10 done
batch 17 of 10 started
batch 17 of 10 done
batch 18 of 10 started
batch 18 of 10 done
batch 19 of 10 started
batch 19 of 10 done
batch 20 of 10 started
batch 20 of 10 done
batch 21 of 10 started
batch 21 of 10 done
batch 22 of 10 started
batch 22 of 10 done
batch 23 of 10 started
batch 23 of 10 done
batch 24 of 10 started
batch 24 of 10 done
batch 25 of 10 started
batch 25 of 10 done
batch 26 of 10 started
batch 26 of 10 done
batch 27 of 10 started
batch 27 of 10 done
batch 28 of 10 started
batch 28 of 10 done
batch 29 of 10 started
batch 29 of 10 done
batch 30 of 10 started
batch 30 of 10 done
batch 31 of 10 started
batch 31 of 10 done
batch 32 of 10 started
batch 32 of 10 done
batch 33 of 10 started
batch 33 of 10 done
batch 34 of

In [9]:
# # pass in batches of 500
# # MADHUKAR: 0 - 5000 
# # SHOBHA: 5000 - 10000
# # RADHIKA: 10000 - 15000

# patch_size = PATCH_SIZE
# batch_size = 500
# MY_NAME = "SHOBHA"
# START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

# for batch in range(10):
#   print("batch {} of 10 started".format(batch + 1))
#   batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
#   extract_image_stats(batch_df, (START + batch_size * batch) + 1, size = patch_size)
#   print("batch {} of 10 done".format(batch + 1))


In [10]:
# # 6 of 10 batches done

# patch_size = PATCH_SIZE
# batch_size = 500
# MY_NAME = "RADHIKA"
# START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

# for batch in range(6, 10):
#   print("batch {} of 10 started".format(batch + 1))
#   batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
#   extract_image_stats(batch_df, (START + batch_size * batch) + 1, size = patch_size)
#   print("batch {} of 10 done".format(batch + 1))


In [11]:
from datetime import datetime
print(datetime.now())

2021-04-03 03:11:39.279832


In [12]:
df_m_test = pd.read_pickle("cwr_nwpr/" + str(2 * PATCH_SIZE) + "m_SRTM/image_stats_" + str(2 * PATCH_SIZE) + "X" + str(2 * PATCH_SIZE) + "_part1")
df_m_test.columns

Index(['jurisdiction_type', 'da_number', 'latitude', 'longitude',
       'cwa_determination', 'Index', 'srtm_stats', 'slope_stats',
       'seasonality_stats', 'recurrence_stats', 'transition_hist'],
      dtype='object')

In [13]:
for item in df_m_test.transition_hist:
    print(item)
# df_m_test.image_stats


{}
{}
{}
{}
{}
{}
{'2': 1.9725490196078432, '5': 1.7058823529411766}
{}
{}
{}
{}
{'10': 0.6862745098039216}
{}
{}
{}
{}
{}
{}
{'5': 1}
{}
{}
{}
{'1': 13.839215686274507, '10': 1.6, '2': 2, '4': 0.2823529411764706, '5': 4.580392156862745, '6': 2, '8': 0.1411764705882353}
{}
{}
{}
{}
{'1': 14.541176470588233, '4': 3.266666666666667, '5': 7.349019607843137, '6': 8.298039215686273}
{}
{}
{'10': 1.211764705882353}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'1': 2.9176470588235293, '5': 5.537254901960785, '6': 0.14901960784313725, '7': 2.768627450980392}
{'5': 1}
{}
{'5': 0.8980392156862745}
{}
{}
{}
{}
